# use r-packages in python

## install packages

In [1]:
#import rpy2.robjects.packages as rpackages

In [2]:
#utils = rpackages.importr('utils')

In [3]:
#utils.chooseCRANmirror(ind=1)

In [4]:
#utils.install_packages('rugarch')

## import packages

In [5]:
# import packages after installed
from rpy2.robjects.packages import importr

In [6]:
rugarch = importr('rugarch')

## model

In [7]:
import pandas as pd
import numpy as np

In [ ]:
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri

In [ ]:
import rpy2.robjects as ro 

In [8]:
fx = pd.read_csv('datasets/DEXUSEU.csv')

In [9]:
fx.head(10)

,observation_date,DEXUSEU,DAILY RETURN
0,2018-01-17,1.2229,NaN
1,2018-01-18,1.2238,1.000736
2,2018-01-19,1.2238,1.000000
3,2018-01-22,1.2230,0.999346
4,2018-01-23,1.2277,1.003843
5,2018-01-24,1.2390,1.009204
6,2018-01-25,1.2488,1.007910
7,2018-01-26,1.2422,0.994715
8,2018-01-29,1.2352,0.994365
9,2018-01-30,1.2390,1.003076


In [10]:
# get daily return
fx = fx['DAILY RETURN'].dropna().values

In [12]:
# setting GARCH model
garch = rugarch.ugarchspec(variance_model = ro.r('list(garchOrder = c(1, 1))'), 
                            mean_model = ro.r('list(armaOrder = c(0, 0))'))
#use ro.r to write r-language

In [54]:
numpy2ri.activate()
pandas2ri.activate()
#fit data to GARCH
garch_fit = rugarch.ugarchfit(spec=garch, data=fx)

In [15]:
garch_fit

R object with classes: ('uGARCHfit',) mapped to:
<RS4 - Python:0x00000205F4E11748 / R:0x00000205EAE7BD50>

In [100]:
# forecast
garch_forecast = rugarch.ugarchforecast(garch_fit, n_ahead=15)

In [101]:
print(garch_forecast)



*------------------------------------*

*       GARCH Model Forecast         *

*------------------------------------*

Model: sGARCH

Horizon: 15

Roll Steps: 0

Out of Sample: 0



0-roll forecast [T0=1970-09-07 07:00:00]:

     Series    Sigma

T+1  0.9997 0.003832

T+2  0.9997 0.003831

T+3  0.9997 0.003830

T+4  0.9997 0.003829

T+5  0.9997 0.003828

T+6  0.9997 0.003827

T+7  0.9997 0.003826

T+8  0.9997 0.003825

T+9  0.9997 0.003823

T+10 0.9997 0.003822

T+11 0.9997 0.003821

T+12 0.9997 0.003820

T+13 0.9997 0.003819

T+14 0.9997 0.003818

T+15 0.9997 0.003817







In [68]:
#check slot names
ro.r.slotNames(garch_fit)

'fit','model'


In [69]:
ro.r.slotNames(garch_forecast)

'forecast','model'


In [102]:
garch_forecast.slots['forecast']

R object with classes: ('list',) mapped to:
<ListVector - Python:0x0000020585A5D348 / R:0x00000205F8F689A8>
[IntVe..., Float..., Float..., Float..., Matrix, Matrix]
  n.ahead: <class 'rpy2.robjects.vectors.IntVector'>
  R object with classes: ('integer',) mapped to:
<IntVector - Python:0x0000020585FB3508 / R:0x00000205F81E1EC8>
[15]
  N: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x0000020585FF6688 / R:0x00000205F83D4C58>
[249.000000]
  n.start: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x0000020585FB3708 / R:0x00000205F11F6498>
[0.000000]
  n.roll: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x0000020585C94788 / R:0x00000205F0BEA490>
[0.000000]
  sigmaFor: <class 'rpy2.robjects.vectors.Matrix'>
  R object with classes: ('matrix',) mapped to:
<Matrix - Python:0x0000020585FF6648 / R:0x0000020586692EB0>
[0.003832, 0.003831, 0.003830, 0.003829, ..., 0.003820, 0.003819, 0.003818, 0.003817]
  seriesFor: <class 'rpy2.robjects.vectors.Matrix'>
  R object with classes: ('matrix',) mapped to:
<Matrix - Python:0x00000205860A52C8 / R:0x0000020586692E00>
[0.999726, 0.999726, 0.999726, 0.999726, ..., 0.999726, 0.999726, 0.999726, 0.999726]

In [104]:
# retreive forecasted value
garch_forecast.slots['forecast'][5] #followed by index

0.999726,0.999726,0.999726,0.999726,...,0.999726,0.999726,0.999726,0.999726


In [105]:
garch_forecast.slots['forecast'].rx2('seriesFor') #use rx2 to indicate name

0.999726,0.999726,0.999726,0.999726,...,0.999726,0.999726,0.999726,0.999726


In [103]:
#convert forecasted values to numpy array
np.array(garch_forecast.slots['forecast'][5]).flatten() #flatten() -> make it 1-dimension

array([0.99972634, 0.99972634, 0.99972634, 0.99972634, 0.99972634,
       0.99972634, 0.99972634, 0.99972634, 0.99972634, 0.99972634,
       0.99972634, 0.99972634, 0.99972634, 0.99972634, 0.99972634])